In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recoMovie').getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.ml.clustering import KMeans
from pyspark.ml.recommendation import ALS

In [0]:
df = sqlContext.sql("SELECT * FROM movielens_ratings_csv")
df.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [0]:
train, test = df.randomSplit([.8,.2])

In [0]:
als=ALS(maxIter=5,regParam=.01, userCol='userId',itemCol='movieId',ratingCol='rating')

In [0]:
model = als.fit(train)

In [0]:
pred = model.transform(test)

In [0]:
pred.show()

+-------+------+------+------------+
movieId|rating|userId| prediction|
+-------+------+------+------------+
 31| 1.0| 4| 2.3412921|
 31| 3.0| 7| 2.5178282|
 31| 4.0| 12| 0.5592197|
 31| 3.0| 14| 2.2778559|
 31| 1.0| 24| -0.10921151|
 31| 2.0| 25| 1.3032799|
 31| 1.0| 27| -0.8044067|
 85| 5.0| 8| 3.4395554|
 85| 1.0| 29| -0.5836527|
 65| 1.0| 4| 1.058292|
 65| 1.0| 19| 3.07905|
 65| 1.0| 22| -0.6934394|
 53| 5.0| 21| 3.5229762|
 78| 1.0| 2| 1.1477137|
 78| 1.0| 4| 1.2264155|
 78| 1.0| 22| 1.8644512|
 78| 1.0| 27| 0.5885926|
 34| 1.0| 28|-0.025837064|
 81| 1.0| 3| 0.83769757|
 81| 2.0| 5| 2.0548432|
+-------+------+------+------------+
only showing top 20 rows

In [0]:
eval = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [0]:
rmse = eval.evaluate(pred)
rmse

Out[18]: 1.763708876264418

In [0]:
su= test.filter(test['userId']==11).select('movieId','userId')
su.show()

+-------+------+
movieId|userId|
+-------+------+
 0| 11|
 21| 11|
 27| 11|
 30| 11|
 32| 11|
 37| 11|
 38| 11|
 47| 11|
 51| 11|
 70| 11|
 75| 11|
 77| 11|
 79| 11|
 99| 11|
+-------+------+

In [0]:
recos=model.transform(su)

In [0]:
recos.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 47| 11| 3.8154173|
 21| 11| 3.550057|
 30| 11| 2.805372|
 38| 11| 2.4145122|
 51| 11| 2.2940242|
 77| 11| 1.853248|
 75| 11| 1.7485428|
 37| 11| 1.244937|
 99| 11| 0.6104208|
 79| 11| -0.1077521|
 32| 11|-0.95315325|
 27| 11| -1.4089017|
 70| 11| -1.7792654|
 0| 11| -2.8929033|
+-------+------+-----------+